<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Compute feature statistics</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [1]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math
import pickle
print(sys.version)

2.7.14 (default, Oct 12 2017, 15:50:02) [GCC]


In [2]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath('/export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [4]:
run /export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/config.py

In [5]:
print config_parameters

{'outputfolder_classifRF_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST/classif_tiles', 'outputfolder_classifRF': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST', 'outputfolder_Logfile': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Log_file', 'permanent_mapset': 'PERMANENT', 'GRASSBIN': '/usr/bin/grass76', 'locationepsg': '31370', 'outputfolder': '/export/miro/cbassine/fusion/RF_fusion/Results_RW', 'list_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/list_tiles', 'pixel_classes_list': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/pixel_classes_list', 'outputfolder_classifRF_csv': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST/classif_csv', 'gisdb': '/export/miro/cbassine/GRASSDATA', 'location': 'WALOUS_31370', 'outputfolder_classfeatures': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features', 'PYTHONLIB': '/usr/bin/python2.7', 'njobs': 6, 'outputfolder_classifRF_valid': '/e

In [6]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [7]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

LESS = -M -I -R 	
BASH_ENV = /usr/share/lmod/7.6.1/init/bash 	
CPU = x86_64 	
BASH_FUNC_module%% = () {  eval $($LMOD_CMD bash "$@") && eval $(${LMOD_SETTARG_CMD:-:} -s sh)
} 	
SSH_CONNECTION = 10.36.1.8 58313 10.36.2.4 22 	
SHELL = /bin/bash 	
XDG_DATA_DIRS = /usr/share 	
HISTSIZE = 1000 	
_ModuleTable003_ = aSIsfSx9LG1wYXRoQT17Ii91c3Ivc2hhcmUvbW9kdWxlcyIsIi91c3Ivc2hhcmUvbG1vZC9tb2R1bGVmaWxlcyIsfSxbInN5c3RlbUJhc2VNUEFUSCJdPSIvdXNyL3NoYXJlL2xtb2QvbW9kdWxlZmlsZXMiLH0= 	
LESS_ADVANCED_PREPROCESSOR = no 	
MANPATH = /usr/lib64/mpi/gcc/openmpi/share/man:/usr/local/man:/usr/share/man 	
JAVA_HOME = /usr/lib64/jvm/jre-1.8.0-openjdk 	
PROFILEREAD = true 	
XDG_RUNTIME_DIR = /run/user/49919 	
JPY_PARENT_PID = 38057 	
PYTHONPATH = /export/apps/os150/gdal-2.2.4/gdal/lib64/python3.6/site-packages:/usr/lib64/grass76/etc/python:/usr/lib64/grass76/etc/python/grass:/usr/lib64/grass76/etc/python/grass/script 	
GISRC = /export/homes/cbassine/.grass7/rc 	
XDG_SESSION_ID = 8 	
DBUS_SESSION_BUS_ADDRESS = uni

** GRASS GIS Python libraries **

In [8]:
from grass_session import Session
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

GRASSBIN: /usr/bin/grass76
GISBASE: /usr/lib64/grass76


** Other functions**

In [9]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for computation of proportion of categorical raster in zones (segments)
from compute_proportion_categorical import CategoStats
# Import function for getting list of class of a raster
from data_prep import data_prep
# Import function the allow sorting strings with number as if they was number (natural order)
from sorting_natural import natural_keys
# Import function that compute ERP on csv file
from EquivalentReferenceProbability import ComputeERPfromCsv
# Import function that replace "empty" values from output .csv to zero
from CsvEmptyValues import CsvChangeEmptyByO

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [10]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
     # Init
    gsetup.init(config_parameters['GISBASE'], config_parameters['gisdb'], config_parameters['location'], mapset)
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [11]:
def GetMapsetsAccess():
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        gscript.run_command('g.mapsets', mapset="BINARIES,OBIA,PIXEL_ORTHO_LC,PIXEL_SENTINEL,STRATA,PERMANENT", operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [12]:
def DefineComputationRegion(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        # Extract the cutline polygon to work on
        condition = 'cat=%s'%tile_cat
        print condition
        gscript.run_command('v.extract', overwrite=True, input='tiles', where=condition, output='tmp_tile')
        # Define computational region based on this polygon
        gscript.run_command('g.region', raster='segmentation') # To get resolution of segmentation 
        gscript.run_command('g.region', flags='a', vector='tmp_tile') # Keep resolution but change extent
        # Mask according to this polygon
        gscript.run_command('r.mask', overwrite=True, vector='tmp_tile')
        # Copy the mask layer
        gscript.run_command('g.copy', overwrite=True, raster='MASK,mask_copy')
        # Print
        return_message += "--> Computational region and MASK defined"
    except:
        return_message += "ERROR: Setting of computional region and MASK failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [13]:
def GetObiaAttributes(tile_cat):    
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:   
        # Decompress and import attribute table of segments of the current tile
        in_path = os.path.join(data['obia_folder'],'CLASSIF','walous_2018_classification_tile_%s.csv'%tile_cat)
        #in_path = os.path.join(data['obia_folder'],'CLASSIF','walous_2018_classification_tile_%s.csv.gz'%tile_cat) #Uncomment if .csv is compressed with Gzip
        #decompress_path = in_path[:-3] #Uncomment if .csv is compressed with Gzip
        #decompress_gzip(in_path, decompress_path) #Uncomment if .csv is compressed with Gzip
        #in_path = decompress_path #Uncomment if .csv is compressed with Gzip
        print in_path
        obia_erp_path = ComputeERPfromCsv(in_path, erp_name="OBIA_ERP", start_index=2)
        print obia_erp_path
        gscript.run_command('db.in.ogr', overwrite=True, input=obia_erp_path, output='attribute')
        # Rename the first column (id -> cat_)
        columns = gscript.read_command('db.columns', table='attribute').split()[1:] #All columns except first ('id')
        print columns
        sql_query = "DROP TABLE IF EXISTS tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.id AS cat_, %s \
        FROM attribute AS a;"%','.join(['a.%s'%col for col in columns])
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "DROP TABLE IF EXISTS attribute;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE attribute AS SELECT * FROM tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        # Remove temp files 
        #os.remove(decompress_path) #Remove decompressed .csv #Uncomment if .csv is compressed with Gzip
        os.remove(obia_erp_path) #Remove .csv with ERP
        # Print
        return_message = "--> Attribute table from OBIA classification imported in GRASS"
    except:
        return_message += "ERROR: Importation of attribute table from OBIA classification failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [14]:
def CutCsv(input_csv, indexcol=False, nb_col_from_end=False):   
    """If provided, 'indexcol' should be a list"""
    """If provided, 'nb_col_from_end' should be an integer"""
    import subprocess, sys
    # Check if at least one option is asked   
    if not indexcol and not nb_col_from_end:
        return "ERROR: At least parameter 'indexcol' or 'nb_col_from_end' should be provided"
        sys.exit()
    # Create a new .csv with only the first and the X lasts columns (X = nb_col_from_end. By default, first and last only)
    path, ext = os.path.splitext(input_csv)
    cut_csv = "%s_cut%s"%(path,ext)
    f = open(cut_csv, "w")
    if nb_col_from_end:
        # Get number of colums in the .csv
        p1 = subprocess.Popen(("head -1 %s"%input_csv).split(), stdout=subprocess.PIPE)
        p2 = subprocess.Popen(("tr ',' '\\n'").split(), stdin=p1.stdout, stdout=subprocess.PIPE)
        p3 = subprocess.Popen(("wc -l").split(), stdin=p2.stdout, stdout=subprocess.PIPE)
        nb_columns = int(p3.communicate()[0])
        index_start = nb_columns-nb_col_from_end
    #Extract first and several specific columns whose indexes are provided in 'indexcol' 
    if indexcol and not nb_col_from_end:
        cut_process =  subprocess.Popen(("cut -d, -f1,%s %s"%(','.join(indexcol),input_csv)).split(), stdout=f)
        cut_process.wait()
    #Extract first and the X last columns according to the value provided in 'nb_col_from_end' 
    if nb_col_from_end and not indexcol:
        cut_process =  subprocess.Popen(("cut -d, -f1,%s-%s %s"%(index_start,nb_columns,input_csv)).split(), stdout=f)
        cut_process.wait()
    #Extract first column, several specific columns ('indexcol') and the X last columns ('nb_col_from_end') 
    if nb_col_from_end and indexcol:
        cut_process =  subprocess.Popen(("cut -d, -f1,%s,%s-%s %s"%(','.join(indexcol),index_start,nb_columns,input_csv)).split(), stdout=f)
        cut_process.wait()
    f.close()
    return cut_csv

def GetObiaNdsm(tile_cat):    
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:   
        # Decompress and import attribute table of segments the current cutline
        in_path = os.path.join(data['obia_folder'],'ATTRIBUTS','walous_2018_stats_tile_%s.csv'%tile_cat)
        print in_path
        #in_path = os.path.join(data['obia_folder'],'ATTRIBUTS','walous_2018_stats_tile_%s.csv.gz'%tile_cat) #Uncomment if .csv is compressed with Gzip
        #decompress_path = in_path[:-3] # Uncomment if .csv is compressed with Gzip
        #decompress_gzip(in_path, decompress_path) #Uncomment if .csv is compressed with Gzip
        #in_path = decompress_path #Uncomment if .csv is compressed with Gzip
        cut_csv = CutCsv(in_path, indexcol=['24',])  #'24' correspond to the position of the nDSM value in statistics csv file
        print cut_csv
        table_name = 'ndsm_mean'
        gscript.run_command('db.in.ogr', overwrite=True, input=cut_csv, output=table_name)
        # Add to 'attribute' table
        sql_query = "DROP TABLE IF EXISTS tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.*,b.mnh2018_mean FROM attribute \
        AS a LEFT JOIN %s AS b ON a.cat_ = b.cat_;"%table_name
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "DROP TABLE IF EXISTS attribute;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE attribute AS SELECT * FROM tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        #os.remove(decompress_path) #Remove decompressed .csv #Uncomment if .csv is compressed with Gzip
        os.remove(cut_csv) #Remove cutted .csv 
        # Print
        return_message = "--> MNH value (nDSM) added to attribute table"
    except:
        return_message += "ERROR: Importation of MNH (nDSM) table failed. Please check for problem."%tile_cat
    return return_message

In [15]:
def GetPixelProportion(basemap, zonemap, prefixcolumn=""):        
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        ### Compute percentage of each class from pixel-based classification
        output_csv = CategoStats(basemap, zonemap, cl_list=pixel_classes_list, colpref=prefixcolumn,
                                 prop=True, mode=True, countnullformode=True)
        print output_csv
        pixel_erp_path = ComputeERPfromCsv(output_csv, erp_name="ortho_lc_ERP", start_index=2)
        ## Join to the attribute table
        table_name = '_'.join(os.path.split(pixel_erp_path)[-1].split(".csv")[0].split('_')[1:])
        print table_name
        gscript.run_command('db.in.ogr', overwrite=True, input=pixel_erp_path, output=table_name)
        proportion_columns = gscript.read_command('db.columns', table=table_name).split()[1:] #All columns except firts ('cat')
        print proportion_columns
        sql_query = "DROP TABLE IF EXISTS tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.*,%s FROM attribute \
        AS a LEFT JOIN %s AS b ON a.cat_ = b.cat_;"%(','.join(['b.%s'%col for col in proportion_columns]),table_name)
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "DROP TABLE IF EXISTS attribute;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE attribute AS SELECT * FROM tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        # Print
        return_message = "--> Proportion of pixel-based classification '%s' computed"%basemap
    except:
        return_message += "ERROR: Computation of proportion of pixel-based classification '%s' failed. Please check for problem."%basemap
    return return_message

In [16]:
def GetPixelModal(basemap, zonemap, prefixcolumn=""):        
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Get mode of pixel-based classification for each segments
        output_csv = CategoStats(basemap, zonemap, cl_list=pixel_classes_list, colpref=prefixcolumn,
                                 prop=False, mode=True, countnullformode=True)
        # Joint attribute from OBIA classification and modal class from pixel-based classification
        mode_raster = '%s_%s_mode'%(basemap,zonemap)
        gscript.run_command('db.in.ogr', overwrite=True, input=output_csv, output=mode_raster)
        sql_query = "DROP TABLE IF EXISTS tmp_a"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.*,b.%s_mode \
        FROM attribute AS a LEFT JOIN %s AS b ON a.cat_ = b.cat_;"%(prefixcolumn,mode_raster)
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "DROP TABLE IF EXISTS attribute;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE attribute AS SELECT * FROM tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        return_message = "--> Modal class of pixel-based classification '%s' computed"%basemap
    except:
        return_message += "ERROR: Computation of modal class of pixel-based classification '%s' failed. Please check for problem."%basemap
    return  return_message

In [17]:
def ExportFeatureCsv(tile_cat):        
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        ## Export attribute table with all features to .csv file
        output_csv = os.path.join(config_parameters['outputfolder_classfeatures'],'features_tile_%s.csv'%tile_cat)
        gscript.run_command('db.select', overwrite=True, sql="SELECT * FROM attribute", separator='comma', output=output_csv)
        CsvChangeEmptyByO(output_csv, sep=',')
        return_message += "--> Attribute table with segment feature exported to csv"
    except:
        return_message += "ERROR: Export attribute table with feature statistics to csv failed. Please check for problem."

    # Return
    return return_message

In [18]:
def DeleteTmpData(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    #Delete layers that are not needed anymore
    list_of_tmp_rast = ['mask_copy',]
    list_of_tmp_vect = ['tmp_tile',]
    try:
        # Remove mask if exists
        if "MASK@%s"%tile_cat in gscript.list_strings(type="raster",mapset=tile_cat):
            gscript.run_command('r.mask', flags='r')
        # Remove layers not needed anymore 
        gscript.run_command('g.remove', quiet=True, flags='f', type='vector', name=','.join(list_of_tmp_vect))    
        gscript.run_command('g.remove', quiet=True, flags='f', type='raster', name=','.join(list_of_tmp_rast))
        return_message += "--> Mask and temporary layers have been deleted"
    except:
        return_message = "WARNING: Something went wrong during the deleting of temporary layer. Please check."
    #Return
    return return_message

In [19]:
def GetTileFeatures(current_tile):
    print current_tile
    start_tile = start_processing() 
    #Declare empty list for saving output messages
    output_message = [] 
    #Launch mapset and create it if not exists
    message = launch_mapset(current_tile)  
    [output_message.append(a) for a in message]
    print "\n".join(message)
    #Add access to other mapset with input data
    message = GetMapsetsAccess()  
    output_message.append(message)
    print message
    #Define computational region and mask
    message = DefineComputationRegion(current_tile) 
    output_message.append(message)
    print message
    #Unzip attributes from OBIA classification and import as SQlite table
    message = GetObiaAttributes(current_tile) 
    output_message.append(message)
    print message
    #Get nDSM value
    message = GetObiaNdsm(current_tile)
    output_message.append(message)
    print message
    #Compute proportion and modal value of ORTHO_LC
    message = GetPixelProportion(data['ortho_lc'][0], "segmentation", prefixcolumn=data['ortho_lc'][0])  
    output_message.append(message)
    print message
    #Compute modal value SENTI_LC
    message = GetPixelModal(data['senti_lc'][0], "segmentation", prefixcolumn=data['senti_lc'][0]) 
    output_message.append(message)
    print message
    #Compute modal value SENT2AGRI
    message = GetPixelModal(data['senti_croptype'][0], "segmentation", prefixcolumn=data['senti_croptype'][0]) 
    output_message.append(message)
    print message
    #Compute modal value MASK CROP 
    message = GetPixelModal(data['binary_crop'][0], "segmentation", prefixcolumn=data['binary_crop'][0]) 
    output_message.append(message)
    print message
    #Compute modal value MASK SIGEC
    message = GetPixelModal(data['binary_agri'][0], "segmentation", prefixcolumn=data['binary_agri'][0]) 
    output_message.append(message)
    print message
    #Compute modal value MASK FOREST (Gembloux)
    message = GetPixelModal(data['binary_forest'][0], "segmentation", prefixcolumn=data['binary_forest'][0]) 
    output_message.append(message)
    print message
    #Compute modal value STRATES
    message = GetPixelModal(data['strata'][0], "segmentation", prefixcolumn=data['strata'][0]) 
    output_message.append(message)
    print message
    #Compute modal value MASK HYDRO (Squelette vecto)
    #message = GetPixelModal(data['binary_hydro'][0], "segmentation", prefixcolumn=data['binary_hydro'][0]) 
    #output_message.append(message)
    #print message    
    #Compute modal value MASK ROADNET (Squelette vecto)
    #message = GetPixelModal(data['binary_roadnet'][0], "segmentation", prefixcolumn=data['binary_roadnet'][0]) 
    #output_message.append(message)
    #print message    
    #Compute modal value MASK BUILT (Squelette vecto)
    #message = GetPixelModal(data['binary_built'][0], "segmentation", prefixcolumn=data['binary_built'][0]) 
    #output_message.append(message)
    #print message    
    #Export GRASS GIS SQlite table to csv
    message = ExportFeatureCsv(current_tile)
    output_message.append(message)
    print message
    #Delete temporary layers and MASK
    message = DeleteTmpData(current_tile)
    output_message.append(message)
    print message
    #Print processing time
    message = print_processing_time(start_tile, "Extraction of classification feature for tile '%s' achieved in "%current_tile)
    output_message.append(message)
    print message
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_feature_extrac_tile_%s.txt"%current_tile),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()
    # Print end
    print print_processing_time(start_tile,"Processing of tile %s achieved in "%current_tile)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [20]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_classfeatures'])
check_create_dir(config_parameters['outputfolder_Logfile'])

The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features' already exists
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Log_file' already exists


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Processing

In [21]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'WALOUS_31370' already exist",
 "'PERMANENT' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PERMANENT'"]

**Import list of tiles to file**

https://www.techcoil.com/blog/how-to-save-and-load-objects-to-and-from-file-in-python-via-facilities-from-the-pickle-module/

In [22]:
import pickle

with open(config_parameters['list_tiles'], 'rb') as outputfile: 
    tile_list = pickle.load(outputfile) 
tile_list

[u'1',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'8',
 u'9',
 u'10',
 u'11',
 u'12',
 u'13',
 u'14',
 u'15',
 u'16',
 u'17',
 u'18',
 u'19',
 u'20',
 u'21',
 u'22',
 u'23',
 u'24',
 u'25',
 u'26',
 u'27',
 u'28',
 u'29',
 u'30',
 u'31',
 u'32',
 u'33',
 u'34',
 u'35',
 u'36',
 u'37',
 u'38',
 u'39',
 u'40',
 u'41',
 u'42',
 u'43',
 u'44',
 u'45',
 u'46',
 u'47',
 u'48',
 u'49',
 u'50',
 u'51',
 u'52',
 u'53',
 u'54',
 u'55',
 u'56',
 u'57',
 u'58',
 u'59',
 u'60',
 u'61',
 u'62',
 u'63',
 u'64',
 u'65',
 u'66',
 u'67',
 u'68',
 u'69',
 u'70',
 u'71',
 u'72',
 u'73',
 u'74',
 u'75',
 u'76',
 u'77',
 u'78',
 u'79',
 u'80',
 u'81',
 u'82',
 u'83',
 u'84',
 u'85',
 u'86',
 u'87',
 u'88',
 u'89',
 u'90',
 u'91',
 u'92',
 u'93',
 u'94',
 u'95',
 u'96',
 u'97',
 u'98',
 u'99',
 u'100',
 u'101',
 u'102',
 u'103',
 u'104',
 u'105',
 u'106',
 u'107',
 u'108',
 u'109',
 u'110',
 u'111',
 u'112',
 u'113',
 u'114',
 u'115',
 u'116',
 u'117',
 u'118',
 u'119',
 u'120',
 u'121',
 u'122',
 u'123',
 

**Get a list of classes in pixel-based classification**

In [23]:
# Import list of class of 'ortho_lc' map
with open(config_parameters['pixel_classes_list'], 'rb') as outputfile: 
    pixel_classes_list = pickle.load(outputfile) 
pixel_classes_list

['0',
 '2',
 '5',
 '6',
 '11',
 '12',
 '31',
 '32',
 '41',
 '42',
 '91',
 '92',
 '93',
 '94',
 '255']

## Extract classification features by segment

In [24]:
# Define number of cores
ncores = 16

In [25]:
# Define current tile
print "Going to process %s tiles on %s cores"%(len(tile_list),ncores)

Going to process 6372 tiles on 16 cores


In [40]:
#1 le 01/10
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
output = p.map(GetTileFeatures, tile_list[3747:3748])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%(ncores))

3748
Location 'WALOUS_31370' already exist
'3748' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/3748'
Access to other mapset added
cat=3748
Working on tile '3748'
--> Computational region and MASK defined
/export/thematic/OBIA_LC_2018/CLASSIF/walous_2018_classification_tile_3748.csv
/export/thematic/OBIA_LC_2018/CLASSIF/walous_2018_classification_tile_3748_ERP.csv
['rf', 'rf_prob_class_11', 'rf_prob_class_12', 'rf_prob_class_13', 'rf_prob_class_21', 'rf_prob_class_22', 'rf_prob_class_41', 'rf_prob_class_42', 'rf_prob_class_5', 'rf_prob_class_6', 'rf_prob_class_902', 'rf_prob_class_905', 'rf_prob_class_906', 'rf_prob_class_911', 'rf_prob_class_912', 'rf_prob_class_913', 'rf_prob_class_941', 'rf_prob_class_942', 'OBIA_ERP']
--> Attribute table from OBIA classification imported in GRASS
/export/thematic/OBIA_LC_2018/ATTRIBUTS/walous_2018_stats_tile_3748.csv
/export/thematic/OBIA_LC_2018/ATTRIBUTS/walous_2018_stats_tile_3748_cut.csv
--> MNH va

'Computation (on 16 cores) achieved in 2 minutes and 34.7 seconds'

**Check log file for ERRORS**

In [35]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_feature_extrac_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

6344 log files in the folder


In [36]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

1 tile(s) faced an ERROR during the processing.

Error on tile 579


In [ ]:
tile_error_list